# 4D ResNet18 Inference

## Kaggle Version

In this notebook we'll use a "4D" ResNet model (strickly speaking: it's still a 3D ResNet) using "4D" images.<br>
We will use 5-fold CV to make the outcome more reliable.

**4D images**
* each image has the following shape: Channel x Width x Height x Depth
* *channel:* the channel represents a specific MRI type. Further, <u>each</u> image contains all four MRI types (i.e., channel=4) --> That is why we say it is "4D"
* *depth:* the depth represents the depth or the number of slices. I tried various values >= 30. Caution: If for a given patient id, the number of images <= the number of slices, we will replace the missing depth slices with zero matrices.
* added some albumentation such as CLAHE, brightness, and CoarseDropout
* removed black pixels (see [Zabir Al Nazi Nabil](https://www.kaggle.com/furcifer/torch-efficientnet3d-for-mri-no-train))

**4D ResNet18**
* copied and edited from [3D-ResNets-PyTorch](https://github.com/kenshohara/3D-ResNets-PyTorch)

**Slicing Strategy**
* take SLICE_NUMBER slices from the middle of each 3D image

#### Imports

In [ ]:
import os
import random
import glob
import re

import pandas as pd

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import cv2

import albumentations as A

from tqdm import tqdm

import wandb

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

#### Seed

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(42)

#### Settings

In [ ]:
PATH = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'

#### Config

In [ ]:
config_dict = dict(
    # Pre-processing
    SLICE_NUMBER = 32, # >= 30
    REMOVE_BLACK_BOUNDARIES = True,
    DICOM=True,
    
    # Albumentation
    RRC_SIZE = 256,
    RRC_MIN_SCALE = 0.85,
    RRC_RATIO = (1., 1.),
    CLAHE_CLIP_LIMIT = 2.0,
    CLAHE_TILE_GRID_SIZE = (8, 8),
    CLAHE_PROB = 0.50,
    BRIGHTNESS_LIMIT = (-0.2,0.2),
    BRIGHTNESS_PROB = 0.40,
    HUE_SHIFT = (-15, 15),
    SAT_SHIFT = (-15, 15),
    VAL_SHIFT = (-15, 15),
    HUE_PROB = 0.64,
    COARSE_MAX_HOLES = 16,
    COARSE_PROB = 0.7,
    
    # K-Fold
    N_SPLITS = 5,
    
    # Training
    BATCH_SIZE = 16,
    
    # Logging
    VERBOSE = False,
    MODELNAME = "01-4D-ResNet-v1"
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### wandb

In [ ]:
wandb.login()
run_name = f'{config_dict["MODELNAME"]}-{str(config_dict["SLICE_NUMBER"])}-{str(config_dict["RRC_SIZE"])}'
run = wandb.init(entity="uzk-wim", project='rsna-miccai-k-fold', config=config_dict, mode="offline", group=run_name)
config = wandb.config

### 1. Load Data

For each person, we were given *four* different MRI types: 
* Fluid Attenuated Inversion Recovery (FLAIR)
* T1-weighted pre-contrast (T1w)
* T1-weighted post-contrast (T1Gd)
* T2-weighted (T2)

We will create 3D images which are composed of each MRI type (4 sequences) where each sequence is composed of *slice_number* (e.g. 50) slices (i.e., depth).<br>
Shape:  *Channel x Width x Height x Depth*


### 1.1 Utilities

#### 1.1.1 Augmentation

In [ ]:
train_transform = A.Compose([
    A.RandomResizedCrop(
        config.RRC_SIZE, config.RRC_SIZE,            
        scale=(config.RRC_MIN_SCALE, 1.0),
        ratio=config.RRC_RATIO,
        p=1.0
    ),
    A.CLAHE(
        clip_limit=config.CLAHE_CLIP_LIMIT,
        tile_grid_size=config.CLAHE_TILE_GRID_SIZE,
        p=config.CLAHE_PROB
    ),
    A.RandomBrightnessContrast(
        brightness_limit=config.BRIGHTNESS_LIMIT,
        p=config.BRIGHTNESS_PROB
    ),
    A.HueSaturationValue(
        hue_shift_limit=config.HUE_SHIFT, 
        sat_shift_limit=config.SAT_SHIFT, 
        val_shift_limit=config.VAL_SHIFT, 
        p=config.HUE_PROB
    ),
    A.CoarseDropout(
        max_holes=config.COARSE_MAX_HOLES,
        p=config.COARSE_PROB),
])

valid_transform = A.Compose([
    A.RandomResizedCrop( 
        config.RRC_SIZE, config.RRC_SIZE,            
        scale=(config.RRC_MIN_SCALE, 1.0),
        ratio=config.RRC_RATIO,
        p=1.0
    )
])

#### 1.1.2 Loading Images

In [ ]:
def dicom_2_image(file, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(file)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        img = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        img = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
    
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    return img

def remove_black_boundaries(img):
    (x, y) = np.where(img > 0)
    if len(x) > 0 and len(y) > 0:
        x_mn = np.min(x)
        x_mx = np.max(x)
        y_mn = np.min(y)
        y_mx = np.max(y)
        if (x_mx - x_mn) > 10 and (y_mx - y_mn) > 10:
            img = img[:,np.min(y):np.max(y)]
    return img

def get_middle_idxs(mri_type):
    # Take SLICE_NUMBER slices from the middle
    threshold = config.SLICE_NUMBER // 2
    minimum_idx = len(mri_type)//2 - threshold if (len(mri_type)//2 - threshold) > 0 else 0
    maximum_idx = len(mri_type)//2 + threshold  # maximum can exceed the index
    step = 1
    if config.VERBOSE:
        print(f"Minimum {minimum_idx}")
        print(f"Maximum {maximum_idx}")
    return minimum_idx, maximum_idx, step

def get_3d_image(mri_type, aug, dicom):
    minimum_idx, maximum_idx, step = get_middle_idxs(mri_type)
    
    # Create list which contains all the 2D images which form the 3D image
    mri_img = []
    for i in range(minimum_idx, maximum_idx, step):
        if i >= len(mri_type):
            break
        file = mri_type[i]
        
        # Load 2D image
        if dicom:
            img = dicom_2_image(file)
        else:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            
        # Remove black boundaries
        if config.REMOVE_BLACK_BOUNDARIES:
            img = remove_black_boundaries(img)
            
        # Augmentation
        if aug:
            transformed = train_transform(image=img)
            img = transformed["image"]
        else:
            transformed = valid_transform(image=img)
            img = transformed["image"]
            
        mri_img.append(np.array(img))
    mri_img = np.array(mri_img).T
    
    # If less than SLICE_NUMBER slices, add SLICE_NUMBER - mri_img.shape[-1] images with only zero values
    if mri_img.shape[-1] < config.SLICE_NUMBER:
        if config.VERBOSE:
            print(f"Current slices: {mri_img.shape[-1]}")
        n_zero = config.SLICE_NUMBER - mri_img.shape[-1]
        mri_img = np.concatenate((mri_img, np.zeros((config.RRC_SIZE, config.RRC_SIZE, n_zero))), axis = -1)
    if config.VERBOSE:
        print(f"Shape of mri_img: {mri_img.shape}")
    return mri_img
    
def load_images(scan_id, aug=True, split="train", dicom=False):
    file_ext = "png"
    if dicom:
        file_ext = "dcm"
    # Ascending sort
    flair = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/FLAIR/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
    t1w = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/T1w/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
    t1wce = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/T1wCE/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
    t2w = sorted(glob.glob(f"{PATH}/{split}/{scan_id}/T2w/*.{file_ext}"), key=lambda f: int(re.sub('\D', '', f)))
    
    if config.VERBOSE:
        print(f"Scan id {scan_id}")
    flair_img = get_3d_image(flair, aug, dicom)
    t1w_img = get_3d_image(t1w, aug, dicom)
    t1wce_img = get_3d_image(t1wce, aug, dicom)
    t2w_img = get_3d_image(t2w, aug, dicom)
    
    img = np.moveaxis(np.array((flair_img, t1w_img, t1wce_img, t2w_img)), 0, 0)
    
    # Normalization
    img = img - img.min()
    img = (img + 1e-5) / (img.max() - img.min() + 1e-5)
    # Return 4D image: ChannelsxWidthxHeightxDepth
    return img

### 1.2 Dataset and Dataloader

In [ ]:
# Load ids and labels and make a stratified 80:20 split
df_test = pd.read_csv(f"{PATH}/sample_submission.csv")

In [ ]:
class RSNADataset(Dataset):
    def __init__(self, ids, labels, split = "train", dicom=False):
        self.ids = ids
        self.labels = labels
        self.split = split
        self.dicom = dicom
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        patient_id = self.ids[idx]
        patient_id = str(patient_id).zfill(5)
        if self.split == "train":
            imgs = load_images(patient_id, aug=True, dicom=self.dicom)
        elif self.split == "valid":
            imgs = load_images(patient_id, aug=False, dicom=self.dicom)
        else:
            imgs = load_images(patient_id, aug=False, split=self.split, dicom=self.dicom)

        if self.split != "test":
            return torch.tensor(imgs, dtype = torch.float32), torch.tensor(self.labels[idx], dtype = torch.long)
        else:
            return torch.tensor(imgs, dtype = torch.float32), torch.tensor(self.ids[idx], dtype = torch.float32)

In [ ]:
test_ds = RSNADataset(df_test["BraTS21ID"].to_numpy(), df_test["MGMT_value"].to_numpy(), split='test', dicom=config.DICOM)
test_dl = DataLoader(test_ds, batch_size=config["BATCH_SIZE"], shuffle=False)

In [ ]:
images, idx = next(iter(test_dl))
print(f"Shape of the batch {images.shape}")
print(f"Batch size: {images.shape[0]}")
print(f"Number of channels each image has: {images.shape[1]}")
print(f"Size of each image is: {images.shape[2]}x{images.shape[3]}")
print(f"Depth of each channel/sequence: {images.shape[-1]}")

### 1.3 Model

In [ ]:
def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
    
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv1x1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = conv3x3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = conv1x1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, block_inplanes, n_input_channels=4, conv1_t_size=7, conv1_t_stride=1, no_max_pool=False,
                 shortcut_type='B', widen_factor=1.0, n_classes=2):
        super().__init__()
        
        # For wider ResNets only
        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]  # 64
        self.no_max_pool = no_max_pool

        # First Block: Convolutional Layer + Batch Normalization + ReLu + Max Pooling
        self.conv1 = nn.Conv3d(n_input_channels, self.in_planes, kernel_size=(conv1_t_size, 7, 7), stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        # Example ResNet18: Each layer consists of two residual blocks and two residual connections
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0], shortcut_type)
        self.layer2 = self._make_layer(block, block_inplanes[1], layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(block, block_inplanes[2], layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(block, block_inplanes[3], layers[3], shortcut_type, stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        # Initialization
        for m in self.modules():
            # He initialization
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2), out.size(3), out.size(4))
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(self._downsample_basic_block, planes=planes * block.expansion, stride=stride)
            else:
                downsample = nn.Sequential(conv1x1x1(self.in_planes, planes * block.expansion, stride), 
                                           nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(in_planes=self.in_planes, planes=planes, stride=stride, downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

### 1.4 Testing

In [ ]:
model_0_path = "01-4D-ResNet-v1-fold-0-loss-0.707.pt"
model_1_path = "01-4D-ResNet-v1-fold-1-loss-0.662.pt"
model_2_path = "01-4D-ResNet-v1-fold-2-loss-0.693.pt"
model_3_path = "01-4D-ResNet-v1-fold-3-loss-0.733.pt"
model_4_path = "01-4D-ResNet-v1-fold-4-loss-0.692.pt"
models = [model_0_path, model_1_path, model_2_path, model_3_path, model_4_path]

In [ ]:
y_hats_models = []
idx_list_models = []

for i in range(5):
    model_path = f"../input/resnetv1/{models[i]}"
    
    # ResNet18
    model = ResNet(BasicBlock, [2, 2, 2, 2], [64, 128, 256, 512])
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    
    y_hats = None
    idx_list = []
    
    for xb, idxb in tqdm(test_dl, desc="Testing"):
        xb = xb.to(device)
        y_hat = model(xb)
        y_hat = F.softmax(y_hat)[:,1].cpu().detach().numpy()
        if y_hats is None:
            y_hats = y_hat
            idx_list = idxb.numpy()
        else:
            y_hats = np.concatenate((y_hats, y_hat), axis=0)
            idx_list = np.concatenate((idx_list, idxb.numpy()), axis=0)
    
    y_hats_models.append(y_hats)
    idx_list_models.append(idx_list)

In [ ]:
assert np.array_equal(idx_list_models[0], idx_list_models[1]) and np.array_equal(idx_list_models[0], idx_list_models[2]) \
and np.array_equal(idx_list_models[0], idx_list_models[3]) and np.array_equal(idx_list_models[0], idx_list_models[4]), "Indices are not equal"

In [ ]:
y_hats_avg = (y_hats_models[0] + y_hats_models[1] + y_hats_models[2] + y_hats_models[3] + y_hats_models[4]) / 5

In [ ]:
df_submission = pd.DataFrame({'BraTS21ID': idx_list_models[0].astype(int), 'MGMT_value': y_hats_avg})
df_submission.to_csv("submission.csv", index=False)
df_submission

In [ ]:
run.finish()

Done